<a href="https://colab.research.google.com/github/sahar7888/AI-Powered-Weather-Based-Activity-Recommender/blob/main/External_Source_data_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **This project demonstrates a dynamic integration of weather data retrieval and AI-driven activity recommendations using Open-Meteo's free weather API and OpenAI's GPT-3.5-turbo model. Users can input the name of a city, and the application fetches the city's current weather conditions, including temperature and a brief description of the weather (e.g., clear skies or cloudy). This information is then passed to the GPT model through a Retrieval-Augmented Generation (RAG) framework, where the AI generates personalized activity suggestions suitable for the current weather. The project is implemented with a user-friendly interface using Gradio, allowing seamless interaction and showcasing the practical application of AI in daily decision-making scenarios, such as planning outdoor activities based on real-time weather insights.**

In [2]:
!pip install openai requests


In [9]:
import openai
import requests

# Set up your OpenAI API key
# openai.api_key =
# Function to get weather data from Open-Meteo API
def get_weather_data(city):
    geocode_url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1"
    geocode_response = requests.get(geocode_url)

    if geocode_response.status_code == 200 and geocode_response.json().get("results"):
        location_data = geocode_response.json()["results"][0]
        latitude = location_data["latitude"]
        longitude = location_data["longitude"]
    else:
        return {"error": "City not found or API issue"}

    weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    weather_response = requests.get(weather_url)

    if weather_response.status_code == 200:
        weather_data = weather_response.json()["current_weather"]
        weather_info = {
            "city": location_data["name"],
            "temperature": weather_data["temperature"],
            "description": "Clear skies" if weather_data["weathercode"] == 0 else "Cloudy or other conditions"
        }
        return weather_info
    else:
        return {"error": "Weather data not available"}

# Function to use GPT Turbo with RAG (external weather data)
def ask_gpt_with_weather_rag(city):
    weather_data = get_weather_data(city)

    if "error" in weather_data:
        return weather_data["error"]

    weather_info_str = (
        f"The current weather in {weather_data['city']} is "
        f"{weather_data['temperature']}°C with {weather_data['description']}."
    )

    prompt = (
        f"You are a helpful assistant. Based on the following weather information:\n\n"
        f"{weather_info_str}\n\n"
        f"Can you recommend an activity suitable for this weather?"
    )

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant knowledgeable about weather and activities."},
            {"role": "user", "content": prompt}
        ]
    )

    return  response.choices[0].message.content

# Example usage
city = "New York"
response = ask_gpt_with_weather_rag(city)
print(response)


With the current weather in New York being 18.1°C and cloudy conditions, it's a great opportunity to enjoy a leisurely walk in the city or visit a museum or art gallery. The cooler temperature can make exploring the city more comfortable, and the overcast sky can create a cozy atmosphere for indoor activities. You could also consider having a picnic in a park or enjoying a warm beverage at a local café.


# **Create Gradio Interface**

In [10]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [15]:
%%writefile app.py

import openai
import requests
import gradio as gr

# Set up your OpenAI API key
# openai.api_key =

# Function to get weather data from Open-Meteo API
def get_weather_data(city):
    geocode_url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1"
    geocode_response = requests.get(geocode_url)

    if geocode_response.status_code == 200 and geocode_response.json().get("results"):
        location_data = geocode_response.json()["results"][0]
        latitude = location_data["latitude"]
        longitude = location_data["longitude"]
    else:
        return {"error": "City not found or API issue"}

    weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    weather_response = requests.get(weather_url)

    if weather_response.status_code == 200:
        weather_data = weather_response.json()["current_weather"]
        weather_info = {
            "city": location_data["name"],
            "temperature": weather_data["temperature"],
            "description": "Clear skies" if weather_data["weathercode"] == 0 else "Cloudy or other conditions"
        }
        return weather_info
    else:
        return {"error": "Weather data not available"}

# Function to use GPT Turbo with RAG (external weather data)
def ask_gpt_with_weather_rag(city):
    weather_data = get_weather_data(city)

    if "error" in weather_data:
        return weather_data["error"]

    weather_info_str = (
        f"The current weather in {weather_data['city']} is "
        f"{weather_data['temperature']}°C with {weather_data['description']}."
    )

    prompt = (
        f"You are a helpful assistant. Based on the following weather information:\n\n"
        f"{weather_info_str}\n\n"
        f"Can you recommend an activity suitable for this weather?"
    )

    response =  openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant knowledgeable about weather and activities."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

# Gradio Interface
def weather_activity(city):
    return ask_gpt_with_weather_rag(city)

interface = gr.Interface(
    fn=weather_activity,
    inputs=gr.Textbox(label="Enter City Name"),
    outputs=gr.Textbox(label="Suggested Activity"),
    title="Weather-Based Activity Recommendation",
    description="Enter a city name to get the current weather and GPT-based activity suggestions."
)

# Launch the Gradio interface
if __name__ == "__main__":
    interface.launch(debug=True)


Writing app.py


In [14]:
%%writefile requirements.txt
openai
requests
gradio


Writing requirements.txt
